# Introduction
This notebook shows how to connect to a crypto exchange and receive a trade feed from it. We then run a strategy using this trade feed to decide on some trades. 

In [ ]:
@file:Repository("https://roboquant.jfrog.io/artifactory/roboquant")
@file:DependsOn("org.roboquant:roboquant-extra:0.8-SNAPSHOT")

In [ ]:
import org.roboquant.feeds.alpaca.*

# Setup
We now create, as usual, a strategy we want to test. There is nothing specific to crypto trading that needs to be taken care of.

In [ ]:
val strategy = TAStrategy(12)

strategy.buy { 
    (ta.cdlMorningStar(it) || ta.cdlMorningDojiStar(it)) && ta.ema(it.close, 5) > ta.ema(it.close, 9)
}

strategy.sell { 
    ta.cdl2Crows(it) && ta.ema(it.close, 3) < ta.ema(it.close, 7)
}


# Feed
Now we create an instance of the AlpacaFeed and subscribe to one or more assets.

In [ ]:
val feed = AlpacaFeed()

In [ ]:
val assets = feed.assets.filter { it.symbol.startsWith("AA") }
assets.summary()

In [ ]:
feed.subscribe(assets)

In [ ]:
val roboquant =  Roboquant(strategy, AccountSummary(), ProgressMetric())

# Run
Al that remains, is to start the run and evaluate the strategy against the feed. We'll run it for 10 minutes. If the time displayed looks off, don't forget that roboquant uses a timezone independent representation for all internal time processing (same as UTC).

In [ ]:
val timeFrame = TimeFrame.nextMinutes(10)
roboquant.run(feed, timeFrame)

# Disconnect
And we disconnect from the exchnage

In [ ]:
feed.disconnect()

In [ ]:
roboquant.broker.account.summary()

In [ ]:
roboquant.logger.summary()